In [4]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras import backend as K
import matplotlib.pyplot as plt
import os
import random
import time

In [5]:
def check_gpu_availability():
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU está disponível.")
    else:
        print("Nenhuma GPU disponível. O código será executado na CPU.")
pd.set_option('display.max_colwidth', None)
check_gpu_availability()

GPU está disponível.


In [6]:
def sliding_window_MLP_Regressor(
    datasetGen, layers, batch, function, iterr, learningRate, outFunc, epocas, shuffle
):
    window_size = int(0.3 * len(datasetGen))
    window_inf = 0
    window_sup = window_size
    window = 1
    global_error = []
    start_time = time.time()  # Início da contagem de tempo       

    while window_sup <= len(datasetGen):
        tf.random.set_seed(0)

        dadosGen = datasetGen[window_inf:window_sup]

        X = pd.concat([dadosGen.shift(1), dadosGen.shift(2), dadosGen.shift(3), dadosGen.shift(4), dadosGen.shift(5)], axis=1)
        y = pd.concat([dadosGen.shift(-5)], axis=1)
        X.dropna(inplace=True)
        y.dropna(subset=["Active Power(kWh)"], inplace=True)
        X = X.to_numpy()
        y = y.to_numpy()
        y = y.flatten()
        X_train, X_temp, y_train, y_temp = train_test_split(
            X, y, test_size=0.2, shuffle=False, stratify=None
        )
        X_test, X_val, y_test, y_val = train_test_split(
        X_temp, y_temp, test_size=0.5, shuffle=False, stratify=None
        )

        # Aplicando o modelo MLP Regressor

        # Conferindo o lag:
        input_sequence_length = len(X[0])
        input_layer = tf.keras.layers.Input(shape=(input_sequence_length,))

        mlp = input_layer  # Inicializar o modelo com a camada de entrada

        for neurons in layers:
            mlp = tf.keras.layers.Dense(units=neurons, activation=function)(mlp)
            mlp = tf.keras.layers.Dropout(0.2)(mlp)

        output_layer = tf.keras.layers.Dense(1, activation=outFunc)(mlp)

        # Definir o modelo
        mlp_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

        early_stopping_callback = tf.keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=iterr, verbose=1, min_delta=0.0001, mode='min'
            ,restore_best_weights=True
        )
        opt = tf.keras.optimizers.Adam(
            learning_rate=learningRate
        )

        # Compilar o modelo
        mlp_model.compile(loss="mean_squared_error", optimizer=opt)
        history = mlp_model.fit(
            x=X_train,
            y=y_train,
            batch_size=int(batch),
            epochs=epocas,
            verbose=1,
            callbacks=[early_stopping_callback],
            validation_data=(X_val,y_val),
            shuffle=shuffle,
            workers=-1,
            use_multiprocessing=True,
        )
        print(mlp_model.summary())
        OutputTest = mlp_model.predict(X_test,workers=-1,use_multiprocessing=True)
        
        # Calculo do erro local(janela)
        local_error = [
            ((previsto - target) ** 2) ** 0.5
            for previsto, target in zip(OutputTest, y_test)
        ]
        # Concatena na lista
        global_error += local_error
        window_inf = window_inf + len(OutputTest)
        window_sup = window_sup + len(OutputTest)
        window += 1
        K.clear_session()

    rmse = np.mean(global_error)
    std = np.std(global_error)

    end_time = time.time()  # Fim da contagem de tempo

    save_time = end_time - start_time  # Tempo total para salvar
    hp_str = "layers_{}__function_{}__batch_{}__iter_{}__LearnR_{}__outFunc_{}__time_{}__lag5".format(
        layers, function, batch, iterr, learningRate,outFunc,save_time
    )

    np.savetxt(
        "resultados_australia_5/" + hp_str + ".csv",
        np.array([rmse, std]),
        delimiter=",",
    )


In [7]:
Data = pd.read_csv("base_australia_diario.csv")
Data.columns = Data.columns.str.strip()
Data_Power = Data["Active Power(kWh)"].dropna()

scaler = MinMaxScaler()

data = Data_Power.values
data = data.reshape(-1, 1)
normalized_data = scaler.fit_transform(data)
Data_Power = pd.Series(
    [value for row in normalized_data for value in row], name="Active Power(kWh)"
)
Data_Power.head()

0    0.894613
1    0.888422
2    0.758117
3    0.903145
4    0.946578
Name: Active Power(kWh), dtype: float64

In [8]:
hidden_layer_sizes = [  
# Cami
    # (5,5),
    # (4,4),
    # (3,3),
    # (16,16),
    # (32,32),
    # (3, 32, 32, 16)
# Eu
    (3,3,3,),
    (4,4,4,),
    (5,10,5,),
    (5,10,10,5),
    (5,10,10,10,5),
    (5,10,25,10,5),
    (8,16,32,16,8,4,2),
]
activation = [
    "sigmoid",
    "tanh",
    "relu",
    "elu",
]
outFuncs = [
    'linear',
    'relu',
    'elu'
]
batch_size = [32]
n_iter_no_change = [10]
learningRates = [0.0001, 0.001]

In [9]:
import pickle
state_file = 'stateIrGen.pkl'
try:
    with open(state_file, 'rb') as f:
        state = pickle.load(f)
        last_completed_iteration = state.get('last_completed_iteration', 0)
except FileNotFoundError:
    state = {}
    last_completed_iteration = 0
for_iter = 0
for layers in hidden_layer_sizes:
    for function in activation:
        for batch in batch_size:
            for iterr in n_iter_no_change:
                for learningRate in learningRates:
                    for outFunc in outFuncs:
                        # Pule iterações já concluídas
                        if last_completed_iteration > 0:
                            last_completed_iteration -= 1
                            continue
                        else:
                            # Atualize o estado após cada iteração
                            for_iter+= 1
                            state['last_completed_iteration'] = for_iter
                            with open(state_file, 'wb') as f:
                                pickle.dump(state, f)
                            print('iteração: ', for_iter)                            
                            print(
                                "Parameters: ",
                                layers,
                                function,
                                batch,
                                iterr,
                                learningRate,
                                outFunc,
                            )
                            sliding_window_MLP_Regressor(
                                Data_Power,
                                layers,
                                batch,
                                function,
                                iterr,
                                learningRate,
                                outFunc,
                                200,
                                False,
                            )

iteração:  1
Parameters:  (3, 3, 3) sigmoid 32 10 0.0001 linear
Train on 778 samples, validate on 98 samples
Epoch 1/200


In [ ]:
from glob import glob


file_list = glob("resultados_australia_5/*")
# file_list = glob("resultados_LOSS/*")

dict_result = {"hyperparameters": [], "rmse": [], "std": []}

for file in file_list:
    array = np.loadtxt(file, delimiter=",")

    dict_result["hyperparameters"].append(file.split("\\")[-1].split(":")[0])
    dict_result["rmse"].append(array[0])
    dict_result["std"].append(array[1])

df = pd.DataFrame(dict_result)
df = df.sort_values(by=["rmse"], ascending=True)
# df = df.sort_values(by=["std"], ascending=True)
pd.set_option('display.max_colwidth', None)
df.head(30)

,hyperparameters,rmse,std
